## Решение

Ниже простое, но эффективное решение задачи, основанное на полносвязной нейросети.

Считываем обучающую выборку.

In [1]:
! wget -O lenta-ru-train.csv https://www.dropbox.com/s/kdupcw1llbdbqwl/lenta-ru-train.csv?dl=0

--2020-04-30 07:02:28--  https://www.dropbox.com/s/kdupcw1llbdbqwl/lenta-ru-train.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.1, 2620:100:601f:1::a27d:901
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/kdupcw1llbdbqwl/lenta-ru-train.csv [following]
--2020-04-30 07:02:28--  https://www.dropbox.com/s/raw/kdupcw1llbdbqwl/lenta-ru-train.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucf57f62cffef30d6f96bf096878.dl.dropboxusercontent.com/cd/0/inline/A20tvMlL9Txd4RVjsSb-itWXLx_oLrbVW22Gb2-A2flUWjD1d6sustxZK1MQ9aujQ-MFoz6ZvGMWtR6_u34KFVSDwN0IEul3jrQsX1ABgElSQ8TgdgidDYNJ9PWpE8jfU8I/file# [following]
--2020-04-30 07:02:28--  https://ucf57f62cffef30d6f96bf096878.dl.dropboxusercontent.com/cd/0/inline/A20tvMlL9Txd4RVjsSb-itWXLx_oLrbVW22Gb2-A2flUWjD1d6sustxZK1MQ9aujQ-MFoz6ZvGMWtR6_u34KFVSDwN0

In [2]:
import pandas as pd

lenta = pd.read_csv('lenta-ru-train.csv',  nrows=100000)
lenta.head()

,title,text,topic,topic_label
0,"Инфляция в январе 2006 года составит 2,6 процента","Глава Росстата Владимир Соколин заявил, что в ...",Экономика,0
1,Никита Михалков учредил День российского кино,У российских кинематографистов появится новый...,Культура,3
2,Марко Матерацци вернется в строй к матчу с ЦСКА,"Медицинский штаб миланского ""Интера"" обнародов...",Спорт,1
3,"Определены лауреаты премии ""Книга года""","Премии ""Книга года"" в 13 номинациях вручены на...",Культура,3
4,Гол Роналду со штрафного спас португальцев от ...,Сборная Португалии сыграла вничью с командой И...,Спорт,1


В данных есть пропуски, но их немного, просто удалим их. Мы будем использовать для предсказания только поле text.

In [0]:
lenta.dropna(inplace=True)

Посмотрим на распределение классов - выборка достаточно сбалансирована, можем не использовать under- и oversampling.

In [0]:
lenta.topic.value_counts()

Экономика          30729
Спорт              24992
Культура           20850
Наука и техника    20553
Бизнес              2876
Name: topic, dtype: int64

Сперва проведем стандартный препроцессинг: лемматизируем наш текст, предварительно удалив все небуквенные символы.

In [5]:
!pip install pymorphy2

     |████████████████████████████████| 51kB 1.7MB/s 
     |████████████████████████████████| 7.1MB 7.9MB/s 


In [0]:
from pymorphy2 import MorphAnalyzer
from functools import lru_cache
from multiprocessing import Pool    
from tqdm import tqdm_notebook as tqdm

In [0]:
#@lru_cache(maxsize=128)
def preprocess(text, M = MorphAnalyzer()):
    tokens = [i for i in text.split() if i.isalpha()]
    lemmas = [M.parse(i)[0].normal_form for i in tokens]
    lemmas_cleaned = [l for l in lemmas if len(l) >= 4]
    return lemmas_cleaned

In [0]:
lenta.text[0][:150]

'Глава Росстата Владимир Соколин заявил, что в январе 2006 года темпы роста цен в России составят 2,6 процента, то есть столько же, сколько они состави'

In [0]:
preprocess(lenta.text[0][:150])

['глава',
 'росстат',
 'владимир',
 'соколиный',
 'январь',
 'темп',
 'рост',
 'цена',
 'россия',
 'составить',
 'есть',
 'столько',
 'сколько',
 'состави']

Применим препроцессинг ко всему датасету:

In [8]:
with Pool(8) as p:
    lemmas = list(tqdm(p.imap(preprocess, lenta['text']), total=len(lenta)))

    
lenta['lemmas'] = lemmas
lenta.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


Process ForkPoolWorker-1:
Process ForkPoolWorker-5:
Process ForkPoolWorker-2:
Process ForkPoolWorker-4:
Process ForkPoolWorker-3:
Process ForkPoolWorker-8:
Process ForkPoolWorker-6:
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-7:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6

KeyboardInterrupt: ignored

In [0]:
lenta = pd.read_csv('lemmatized.csv')

Отлично, но дальше мы хотим использовать CountVectorizer (мешок слов), а ему на вход нужен не список слов, а строка, поэтому склеим слова через пробел:

In [10]:
lenta['lemmas'] = lenta.lemmas.apply(lambda x:  ' '.join([i for i in x]))
lenta.head()

TypeError: ignored

In [0]:
lenta[['lemmas', 'topic']].to_csv('lemmatized.csv', index=False)

## avg w2v

In [0]:
! wget http://vectors.nlpl.eu/repository/20/187.zip

--2020-04-30 09:14:00--  http://vectors.nlpl.eu/repository/20/187.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.225
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.225|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2692389554 (2.5G) [application/zip]
Saving to: ‘187.zip’

100%[====================================>] 2,692,389,554 11.9MB/s   in 4m 37s 

2020-04-30 09:18:38 (9.26 MB/s) - ‘187.zip’ saved [2692389554/2692389554]



In [0]:
import zipfile
with zipfile.ZipFile('187.zip', 'r') as zip_ref:
    zip_ref.extractall('w2v_model/')

In [0]:
import gensim 
from gensim.models import word2vec 


model_path = 'w2v_model/model.model'

model_ru = gensim.models.KeyedVectors.load(model_path)

In [0]:
model_ru['монеточка'][:10]

array([-0.24568336,  1.2996082 ,  0.57665306,  1.8290797 ,  0.39956525,
       -0.02574062, -0.16593023,  0.5755158 , -0.97565746,  2.4338129 ],
      dtype=float32)

In [0]:
len(model_ru['монеточка'])

300

In [0]:
import numpy as np
def get_avg_embedding(text, model = model_ru):
    if isinstance(text, str):
        words = text.split()
        c = 0
        vec = np.zeros(300)
        for w in words:
            vec += model[w]
        if len(words):
            vec = vec / len(words)
            return vec
        else:
            return np.zeros(300)
    return np.zeros(300)

In [0]:
get_avg_embedding('я иду домой есть пирожки')

array([ 0.09322592,  0.96203394,  0.59418345, -0.27382224,  0.23096753,
       -0.94382048,  0.321128  , -0.41682359, -1.15042249,  1.31867643,
       -1.18423665, -0.12895398,  0.36244778,  1.04054665, -1.39815475,
        0.05231342,  1.95948308,  0.88611201, -0.72135888,  0.84086897,
        0.65109519,  0.0196041 , -1.15615925, -0.93316307, -1.48000474,
        1.52881389,  0.97408531,  1.01387843,  0.39935729,  0.08648463,
        0.84739794,  0.4495558 , -0.23355194,  0.26153612, -0.91979943,
        0.70217758, -0.09281321, -0.34745254, -1.2370256 , -0.85935306,
        0.18128468,  0.89549725, -0.28527589, -0.08020033, -0.06996629,
       -0.75018733, -0.16368707,  0.47830083, -1.51660514, -0.52826488,
       -0.15057356, -0.1468262 , -0.39582176, -0.57264404, -0.69365329,
        0.34750734, -0.25069273, -0.71260436, -0.96816917,  0.66779003,
       -0.97526909,  0.99993023,  0.12624577,  0.51852878, -0.62567329,
       -1.72503577,  1.43138344,  0.25398048, -0.84152109, -0.74

In [0]:
with Pool(8) as p:
    vecs = list(tqdm(p.imap(get_avg_embedding, lenta['lemmas']), total=len(lenta)))

    
lenta['avg_vec'] = vecs
lenta.head()

,title,text,topic,topic_label,lemmas,avg_vec
0,"Инфляция в январе 2006 года составит 2,6 процента","Глава Росстата Владимир Соколин заявил, что в ...",Экономика,0,глава росстат владимир соколиный январь темп р...,"[0.16688824101952435, -0.2846602177884005, 0.7..."
1,Никита Михалков учредил День российского кино,У российских кинематографистов появится новый...,Культура,3,российский кинематографист появиться новый про...,"[1.0043369824281678, 0.03792253824380728, -0.0..."
2,Марко Матерацци вернется в строй к матчу с ЦСКА,"Медицинский штаб миланского ""Интера"" обнародов...",Спорт,1,медицинский штаб миланский обнародовать новое ...,"[1.210477464125458, 0.047426638967660535, 0.68..."
3,"Определены лауреаты премии ""Книга года""","Премии ""Книга года"" в 13 номинациях вручены на...",Культура,3,премия номинация вручить московский международ...,"[0.943377378694329, -1.238564177272246, 0.0400..."
4,Гол Роналду со штрафного спас португальцев от ...,Сборная Португалии сыграла вничью с командой И...,Спорт,1,сборная португалия сыграть вничью команда испа...,"[0.5890408798130518, 0.5553445452728223, 0.297..."


In [0]:
x_train, x_test, y_train, y_test = train_test_split(lenta.avg_vec, lenta.topic_label)

In [0]:
x_train = list(x_train)
y_train = list(y_train)

In [0]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial', max_iter=400)
LR.fit(x_train, y_train)

/home/jnb/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=400,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
x_test = list(x_test)
y_test = list(y_test)

In [0]:
from sklearn.metrics import classification_report

y_pred = LR.predict(x_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.90      0.96      0.93      7734
           1       0.99      0.99      0.99      6217
           2       0.94      0.94      0.94      4979
           3       0.97      0.97      0.97      5344
           4       0.51      0.16      0.24       726

    accuracy                           0.94     25000
   macro avg       0.86      0.80      0.81     25000
weighted avg       0.93      0.94      0.94     25000



In [0]:
y_pred[:10]

array([2, 3, 1, 2, 3, 0, 2, 0, 1, 3])

Нам нужно проделать все те же преобразования с текстом, что и для обучающей выборки:

In [0]:
! wget -O lenta-ru-test.csv https://www.dropbox.com/s/9av90tgqob1slnj/lenta-ru-test.csv?dl=0

In [0]:
test_data = pd.read_csv('lenta-ru-test.csv')

test_data.head()

,title,text
0,Сборная России выиграла чемпионат мира по футб...,В южноафриканском Кейптауне победой сборной Ро...
1,Анна Курникова снова проиграла в первом раунде,"Анна Курникова, проиграв испанке Кончите Марти..."
2,"Новым главным тренером ""Спартака"" стал Станисл...",Новым главным тренером московского футбольного...
3,Майкл Дуглас совершил неожиданный визит на Кубу,Известный голливудский актер Майкл Дуглас сове...
4,Кадыров снялся в короткометражке «Волшебный гр...,Глава Чеченской Республики Рамзан Кадыров снял...


In [0]:
with Pool(8) as p:
    lemmas = list(tqdm(p.imap(preprocess, test_data['text']), total=len(test_data)))

    
test_data['lemmas'] = lemmas
test_data.head()

,title,text,lemmas
0,Сборная России выиграла чемпионат мира по футб...,В южноафриканском Кейптауне победой сборной Ро...,"[южноафриканский, кейптаун, победа, сборный, р..."
1,Анна Курникова снова проиграла в первом раунде,"Анна Курникова, проиграв испанке Кончите Марти...","[анна, проиграть, испанка, кончить, мартинес, ..."
2,"Новым главным тренером ""Спартака"" стал Станисл...",Новым главным тренером московского футбольного...,"[новый, главное, тренер, московский, футбольны..."
3,Майкл Дуглас совершил неожиданный визит на Кубу,Известный голливудский актер Майкл Дуглас сове...,"[известный, голливудский, актёр, майкл, дуглас..."
4,Кадыров снялся в короткометражке «Волшебный гр...,Глава Чеченской Республики Рамзан Кадыров снял...,"[глава, чеченский, республика, рамзан, кадыров..."


In [0]:
test_data['lemmas'] = test_data.lemmas.apply(lambda x:  ' '.join([i for i in x]))
test_data.to_csv('lemmatized_test.csv', index=False)

In [0]:
test_data = pd.read_csv('lemmatized_test.csv')
test_data.head(2)

,title,text,lemmas
0,Сборная России выиграла чемпионат мира по футб...,В южноафриканском Кейптауне победой сборной Ро...,южноафриканский кейптаун победа сборный россия...
1,Анна Курникова снова проиграла в первом раунде,"Анна Курникова, проиграв испанке Кончите Марти...",анна проиграть испанка кончить мартинес первое...


In [0]:
with Pool(8) as p:
    vecs = list(tqdm(p.imap(get_avg_embedding, test_data['lemmas']), total=len(lenta)))

preds = LR.predict(list(vecs))
preds[:10]

array([1, 1, 1, 3, 3, 0, 0, 1, 2, 0])

In [0]:
preds = pd.DataFrame(preds)
preds['index'] = preds.index
preds['topic_label'] = preds[0]
preds[['index', 'topic_label']].to_csv('solution_w2v.csv', index = None)
preds.head(3)

,0,index,topic_label
0,1,0,1
1,1,1,1
2,1,2,1


### Test accuracy: 0.94171

## fastText

In [11]:
lenta.head()

,lemmas,topic
0,глава росстат владимир соколиный январь темп р...,Экономика
1,российский кинематографист появиться новый про...,Культура
2,медицинский штаб миланский обнародовать новое ...,Спорт
3,премия номинация вручить московский международ...,Культура
4,сборная португалия сыграть вничью команда испа...,Спорт


In [12]:
! git clone https://github.com/facebookresearch/fastText.git
! pip3 install fastText/.

Cloning into 'fastText'...
remote: Enumerating objects: 3814, done.
remote: Total 3814 (delta 0), reused 0 (delta 0), pack-reused 3814
Receiving objects: 100% (3814/3814), 8.19 MiB | 31.43 MiB/s, done.
Resolving deltas: 100% (2401/2401), done.
Processing ./fastText
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3014028 sha256=e26570dd5e1b048f3c9208488b2f920827a185108759d57ad206f84e1d27a178
  Stored in directory: /tmp/pip-ephem-wheel-cache-prmz1xlz/wheels/a1/9f/52/696ce6c5c46325e840c76614ee5051458c0df10306987e7443
Successfully built fasttext


In [0]:
import fasttext

In [17]:
import numpy as np
from sklearn.model_selection import train_test_split
X = lenta.lemmas.tolist()
y = lenta.topic.tolist()

X, y = np.array(X), np.array(y)

# разбиваем на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.33)
print ("total train examples %s" % len(y_train))
print ("total test examples %s" % len(y_test))

total train examples 67000
total test examples 33000


In [0]:
with open('data.train.txt', 'w+') as outfile:
    for i in range(len(X_train)):
        outfile.write('__label__' + y_train[i] + ' '+ X_train[i] + '\n')
    

with open('test.txt', 'w+') as outfile:
    for i in range(len(X_test)):
        outfile.write('__label__' + y_test[i] + ' ' + X_test[i] + '\n')

In [0]:
classifier = fasttext.train_supervised('data.train.txt')
result = classifier.test('test.txt')

In [22]:
print('P@1:', result[1])
print('R@1:', result[2])

P@1: 0.9702727272727273
R@1: 0.9702727272727273


In [37]:
test_data = pd.read_csv('lemmatized_test.csv').fillna('')
test_data = list(test_data['lemmas'])

preds = classifier.predict(test_data)
preds = [i[0][9:] for i in preds[0]]
preds[:5]

['Спорт', 'Спорт', 'Спорт', 'Культура', 'Культура']

In [0]:
def f(x):
    m = {'Бизнес': 4,
         'Культура':3,
         'Наука и техника':2,
         'Спорт':1,
         'Экономика':0}
    return m[x]

preds = [f(i) for i in preds]

In [40]:
preds = pd.DataFrame(preds)
preds['index'] = preds.index
preds['topic_label'] = preds[0]
preds[['index', 'topic_label']].to_csv('solution_ft.csv', index = None)
preds.head(3)

,0,index,topic_label
0,1,0,1
1,1,1,1
2,1,2,1


## simple NN

К лемматизированному датасету применяем мешок слов:

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(min_df=20, lowercase=True)
vectorizer.fit(lenta.lemmas)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=20,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [0]:
vectors = vectorizer.transform(lenta.lemmas).toarray()

Кроме того, мы решаем задачу классифиакции на 5 классов, и метки нужно будет подавать на вход модели в виде one-hot-encoded вектора ([0,0,1,0,0] для класса 3, например). Преобразуем наши метки:

In [0]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

encoder = LabelEncoder()
encoder.fit(lenta.topic)
encoded_Y = encoder.transform(lenta.topic)
dummy_y = np_utils.to_categorical(encoded_Y)

Using TensorFlow backend.


In [0]:
encoder.classes_

array(['Бизнес', 'Культура', 'Наука и техника', 'Спорт', 'Экономика'],
      dtype=object)

Все готово: и тексты, и метки преобразованы к нужному формату, разбиваем наш датасет на train и test для обучения:

In [0]:
from sklearn.model_selection import train_test_split

x_train,  x_test,y_train, y_test = train_test_split(vectors, dummy_y, test_size=0.3)
x_train.shape, x_test.shape

((70000, 17720), (30000, 17720))

In [0]:
y_train[0]

array([0., 0., 0., 0., 1.], dtype=float32)

Попробуем простую полносвязную сеть:

In [0]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

In [0]:
input_dim = x_train.shape[1]  # Number of features

model = Sequential()
model.add(Dense(30, input_dim=input_dim, activation='relu'))
model.add(Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
history = model.fit(x_train, y_train,
                     epochs=20,
                     verbose=True,
                     validation_data=(x_test, y_test),
                     batch_size=1000)

Train on 70000 samples, validate on 30000 samples
Epoch 1/20
70000/70000 [==============================] - 9s 131us/step - loss: 0.4286 - accuracy: 0.9138 - val_loss: 0.1832 - val_accuracy: 0.9506
Epoch 2/20
70000/70000 [==============================] - 9s 131us/step - loss: 0.1321 - accuracy: 0.9627 - val_loss: 0.1439 - val_accuracy: 0.9566
Epoch 3/20
70000/70000 [==============================] - 9s 130us/step - loss: 0.0875 - accuracy: 0.9756 - val_loss: 0.1346 - val_accuracy: 0.9572
Epoch 4/20
70000/70000 [==============================] - 9s 130us/step - loss: 0.0625 - accuracy: 0.9839 - val_loss: 0.1330 - val_accuracy: 0.9580
Epoch 5/20
70000/70000 [==============================] - 9s 133us/step - loss: 0.0463 - accuracy: 0.9891 - val_loss: 0.1364 - val_accuracy: 0.9567
Epoch 6/20
70000/70000 [==============================] - 9s 131us/step - loss: 0.0347 - accuracy: 0.9930 - val_loss: 0.1413 - val_accuracy: 0.9566
Epoch 7/20
70000/70000 [==============================] - 9s 1

KeyboardInterrupt: 

Как видим, уже на второй эпохе качество очень высокое. Дальше можно подобрать параметры сети, добавить слоев или инициализировать тексты не вектором мешка слов, а более умными предобученными эмбедингами.

Теперь отправим наше решение в Kaggle:

In [0]:
test_vectors = vectorizer.transform(test_data.lemmas).toarray()

Получаем предсказания для всех объектов тестовой выборки:

In [0]:
preds = model.predict_classes(test_vectors)

Чтобы отправить решение на Kaggle, нужно раскодировать метки обратно в названия классов, в соответствиии с тем, как из закодировал LabelEncoder:

In [0]:
preds[:10]

array([3, 3, 3, 1, 1, 4, 4, 3, 2, 4])

In [0]:
def f(x):
    m = {0: 'Бизнес',
         1: 'Культура',
         2: 'Наука и техника',
         3:'Спорт',
         4:'Экономика'}
    return m[x]

f(3)

'Спорт'

In [0]:
test_data['label'] = [f(pred) for pred in preds]
test_data[['text', 'label']].head()

,text,label
0,В южноафриканском Кейптауне победой сборной Ро...,Спорт
1,"Анна Курникова, проиграв испанке Кончите Марти...",Спорт
2,Новым главным тренером московского футбольного...,Спорт
3,Известный голливудский актер Майкл Дуглас сове...,Культура
4,Глава Чеченской Республики Рамзан Кадыров снял...,Культура


In [0]:
def f(x):
    m = {'Бизнес': 4,
         'Культура':3,
         'Наука и техника':2,
         'Спорт':1,
         'Экономика':0}
    return m[x]

f('Спорт')

1

In [0]:
test_data['label'] = test_data.label.map(f)
test_data[['text', 'label']].head()

,text,label
0,В южноафриканском Кейптауне победой сборной Ро...,1
1,"Анна Курникова, проиграв испанке Кончите Марти...",1
2,Новым главным тренером московского футбольного...,1
3,Известный голливудский актер Майкл Дуглас сове...,3
4,Глава Чеченской Республики Рамзан Кадыров снял...,3


Сделаем столбец id с номерами объектов, в соответствии с форматом посылки в Kaggle и сохраним в файл:

In [0]:
test_data['index'] = test_data.index
test_data['topic_label'] = test_data.label
test_data[['index', 'topic_label']].to_csv('solution.csv', index = None)

### Test accuracy: 0.95334